In [1]:
!pip install --quiet llama-index-core llama-index-utils-workflow llama-index-llms-openai llama-index-graph-stores-neo4j 

In [1]:
import os
from llama_index.llms.openai import OpenAI
from neo4j.exceptions import CypherSyntaxError

from llama_index.core import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List, Literal, Union, Optional

from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context,
)

/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore

graph_store = Neo4jPropertyGraphStore(
    username="recommendations",
    password="recommendations",
    database="recommendations",
    url="neo4j+s://demo.neo4jlabs.com:7687",
    enhanced_schema=True,
    create_indexes=False,
    timeout=10
)

In [3]:

os.environ["OPENAI_API_KEY"] = "sk-"
"""
llm = OpenAI(model="gpt-4o-2024-11-20", temperature=0)
llm_fast = OpenAI(model="gpt-4o-2024-11-20", temperature=0)
"""

'\nllm = OpenAI(model="gpt-4o-2024-11-20", temperature=0)\nllm_fast = OpenAI(model="gpt-4o-2024-11-20", temperature=0)\n'

In [4]:
from llama_index.llms.gemini import Gemini

llm = Gemini(
    model="models/gemini-2.0-flash-exp",
    api_key=""
)

llm_fast = Gemini(
    model="models/gemini-1.5-flash",
    api_key=""
)


In [5]:
class Guardrail(BaseModel):
    """Guardrail"""

    decision: Literal["movie", "end"] = Field(
        description="Decision on whether the question is related to movies"
        
    )
guardrails_system_prompt = """As an intelligent assistant, your primary objective is to decide whether a given question is related to movies or not.
If the question is related to movies, output "movie". Otherwise, output "end".
To make this decision, assess the content of the question and determine if it refers to any movie, actor, director, film industry,
or related topics. Provide only the specified output: "movie" or "end"."""
# Refine Prompt
chat_refine_msgs = [
    (
        "system",
        guardrails_system_prompt,
    ),
    ("user", "The question is: {question}"),
]
guardrails_template = ChatPromptTemplate.from_messages(chat_refine_msgs)

In [6]:
class SubqueriesOutput(BaseModel):
    """Defines the output format for transforming a question into parallel-optimized retrieval steps."""
    
    plan: List[List[str]] = Field(description=("""A list of query groups where:
        - Each group (inner list) contains queries that can be executed in parallel
        - Groups are ordered by dependency (earlier groups must be executed before later ones)
        - Each query must be a specific information retrieval request
        - No reasoning or comparison tasks, only data fetching queries"""))

subqueries_system = """You are a query planning optimizer. Your task is to break down complex questions into efficient, parallel-optimized retrieval steps. Focus ONLY on information retrieval queries, not analysis or reasoning steps.

Key Requirements:
- Group queries that can be executed in parallel into the same list
- Order groups based on data dependencies
- Include ONLY specific information retrieval queries
- Exclude reasoning tasks, comparisons, or analysis steps
- Prioritize queries that can be executed first and in parallel

For simple, directly answerable questions, return a single query in a single group.

Example 1:
User: "What was the impact of the 2008 financial crisis on Bank of America's stock price and employee count?"
Assistant: [
    # Group 1 - These can be fetched in parallel
    [
        "What was Bank of America's stock price history from 2007 to 2009?",
        "What was Bank of America's total employee count in 2007?",
        "What was Bank of America's total employee count in 2009?"
    ]
]

Example 2:
User: "Compare the performance of Tesla's Model 3 with BMW's competing models in terms of range and acceleration."
Assistant: [
    # Group 1 - Basic specs can be fetched in parallel
    [
        "What is the EPA range of the Tesla Model 3?",
        "What is the 0-60 mph acceleration time of the Tesla Model 3?",
        "What BMW models compete directly with the Tesla Model 3?"
    ],
    # Group 2 - Depends on knowing competing models from group 1
    [
        "What is the EPA range of each identified BMW competitor model?",
        "What is the 0-60 mph acceleration time of each identified BMW competitor model?"
    ]
]

Remember:
- Focus on data retrieval only
- Maximize parallel execution opportunities
- Maintain necessary sequential ordering
- Keep queries specific and self-contained
- Prioritize independent queries first"""

query_decompose_msgs = [
    ("system", subqueries_system),
    ("user", "{question}")
]

subquery_template = ChatPromptTemplate.from_messages(query_decompose_msgs)

In [7]:
def guardrails_step(question):
    guardrails_output = (
        llm_fast.as_structured_llm(Guardrail)
        .complete(guardrails_template.format(question=question))
        .raw
    ).decision
    if guardrails_output == 'end':
        context = "The question is not about movies or their case, so I cannot answer this question"
        return {"next_event": "generate_final_answer", "arguments": {"context": context, "question": question}}
    # Refactor into separate step
    queries_output = (
        llm.as_structured_llm(SubqueriesOutput)
        .complete(subquery_template.format(question=question))
        .raw
    ).plan
    return {"next_event": "generate_cypher", "arguments": {"plan": queries_output, "question": question}}


In [8]:
from llama_index.core.schema import TextNode
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(model="text-embedding-3-small")


examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Schindler's List",
        "query": "MATCH (m:Movie {title: 'Schindler's List'})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

few_shot_nodes = []
for line in examples:
    few_shot_nodes.append(TextNode(text=f"{{'query':{line['query']}, 'question': {line['question']}))"))

few_shot_index = VectorStoreIndex(few_shot_nodes, embed_model=embed_model)
few_shot_retriever = few_shot_index.as_retriever(similarity_top_k=5)


def get_fewshots(question):
    return [el.text for el in few_shot_retriever.retrieve(question)]

/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [9]:
generate_system = """Given an input question, convert it to a Cypher query. No pre-amble.
Do not wrap the response in any backticks or anything else. Respond with a Cypher statement only!"""

generate_user = """You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.
Do not wrap the response in any backticks or anything else. Respond with a Cypher statement only!
Here is the schema information
{schema}

Below are a number of examples of questions and their corresponding Cypher queries.

{fewshot_examples}

User input: {question}
Cypher query:"""

generate_cypher_msgs = [
    (
        "system",
        generate_system,
    ),
    ("user", generate_user),
]

text2cypher_prompt = ChatPromptTemplate.from_messages(generate_cypher_msgs)

In [10]:
schema = graph_store.get_schema_str(exclude_types=["Actor", "Director"])

async def generate_cypher(subquery):
    fewshot_examples = get_fewshots(subquery)
    resp = await llm.achat(text2cypher_prompt.format_messages(question=subquery, schema=schema, fewshot_examples=fewshot_examples))
    return resp.message.content

In [11]:
validate_cypher_system = """You are a specialized parser focused on analyzing Cypher query statements to extract node property filters. Your task is to identify and extract properties used in WHERE clauses and pattern matching conditions, but only when they contain explicit literal values.

For each Cypher statement, you should:

1. Identify node labels and their associated property filters
2. Extract the property key and its matching literal value
3. Format the output as a JSON object containing a "filters" array with objects having:
   - node_label: The label of the node (e.g., "Person")
   - property_key: The property name being filtered (e.g., "age")
   - property_value: The literal value being matched (e.g., 30)

Rules for extraction:
- Only extract filters that match against literal values (strings, numbers, booleans)
- Include property filters in MATCH patterns (e.g., (p:Person {name: 'John'}))
- Include property filters in WHERE clauses with literal values (e.g., WHERE p.age > 30)
- Handle both simple equality and comparison operators with literal values
- Ignore property-to-property comparisons (e.g., WHERE m1.rating = m2.rating)
- Ignore variable references or dynamic values
- Valid literal values are:
  * Quoted strings (e.g., 'John', "London")
  * Numbers (e.g., 30, 42.5)
  * Booleans (true, false)

Example input 1:
MATCH (p:Person {name: 'John'})-[:KNOWS]->(f:Friend)
WHERE p.age > 30 AND f.city = 'London' AND f.salary = m.salary

Example output 1:
{
    "filters": [
        {
            "node_label": "Person",
            "property_key": "name",
            "property_value": "John"
        },
        {
            "node_label": "Person",
            "property_key": "age",
            "property_value": 30
        },
        {
            "node_label": "Friend",
            "property_key": "city",
            "property_value": "London"
        }
    ]
}

Example input 2:
MATCH (m1:Movie {title: 'Matrix'}), (m2:Movie)
WHERE m1.rating > m2.rating AND m1.year = 1999

Example output 2:
{
    "filters": [
        {
            "node_label": "Movie",
            "property_key": "title",
            "property_value": "Matrix"
        },
        {
            "node_label": "Movie",
            "property_key": "year",
            "property_value": 1999
        }
    ]
}

Note how property-to-property comparisons (f.salary = m.salary, m1.rating > m2.rating) are ignored in the output."""

validate_cypher_user = """Cypher statement: {cypher}"""

validate_cypher_msgs = [
    (
        "system",
        validate_cypher_system,
    ),
    ("user", validate_cypher_user),
]

validate_cypher_prompt = ChatPromptTemplate.from_messages(validate_cypher_msgs)

class Property(BaseModel):
    """
    Represents a filter condition based on a specific node property in a graph in a Cypher statement.
    """

    node_label: str = Field(
        description="The label of the node to which this property belongs."
    )
    property_key: str = Field(description="The key of the property being filtered.")
    property_value: str = Field(
        description="The value that the property is being matched against."
    )


class ValidateCypherOutput(BaseModel):
    """
    Represents the applied filters of a Cypher query's output.
    """
    filters: Optional[List[Property]] = Field(
        description="A list of property-based filters applied in the Cypher statement."
    )

In [12]:
from llama_index.graph_stores.neo4j import CypherQueryCorrector, Schema

# Cypher query corrector is experimental
corrector_schema = [
    Schema(el["start"], el["type"], el["end"])
    for el in graph_store.get_schema().get("relationships")
]
cypher_query_corrector = CypherQueryCorrector(corrector_schema)

In [13]:
def validate_cypher(question, cypher):
    """
    Validates the Cypher statements and maps any property values to the database.
    """
    errors = []
    mapping_errors = []
    # Check for syntax errors
    try:
        graph_store.structured_query(f"EXPLAIN {cypher}")
    except CypherSyntaxError as e:
        errors.append(e.message)
    # Experimental feature for correcting relationship directions
    corrected_cypher = cypher_query_corrector(cypher)
    if not corrected_cypher:
        errors.append("The generated Cypher statement doesn't fit the graph schema")
    # Use LLM for mapping for values
    llm_output =   (
        llm.as_structured_llm(ValidateCypherOutput)
        .complete(validate_cypher_prompt.format(cypher=cypher))
        .raw
    )
    if llm_output.filters:
        for filter in llm_output.filters:
            # Do mapping only for string values
            try:
                if (
                    not [
                        prop
                        for prop in graph_store.get_schema()["node_props"][
                            filter.node_label
                        ]
                        if prop["property"] == filter.property_key
                    ][0]["type"]
                    == "STRING"
                ):
                    continue
            except: # if property is hallucinated/doesn't exist in the schema # ToDo handle it better
                continue
            mapping = graph_store.structured_query(
                f"MATCH (n:{filter.node_label}) WHERE toLower(n.`{filter.property_key}`) = toLower($value) RETURN 'yes' LIMIT 1",
                {"value": filter.property_value},
            )
            if not mapping:
                mapping_errors.append(
                    f"Could not find node in graph with label '{filter.node_label}' where property '{filter.property_key}' equals '{filter.property_value}'. "
                    f"Without this information, I cannot provide a complete answer to your question. "
                    f"If you meant something else, please rephrase your question or verify the specific {filter.property_key} you're asking about. "
                    f"Would you like to try with a different {filter.property_key} value?"
                )
    if mapping_errors:
        next_action = "end"
    elif errors:
        next_action = "correct_cypher"
    else:
        next_action = "execute_cypher"

    return {
        "next_action": next_action,
        "cypher_statement": corrected_cypher,
        "cypher_errors": errors,
        "mapping_errors": mapping_errors,
        "steps": ["validate_cypher"],
    }

In [14]:
correct_cypher_system = """You are a Cypher expert reviewing a statement written by a junior developer. 
You need to correct the Cypher statement based on the provided errors. No pre-amble."
Do not wrap the response in any backticks or anything else. Respond with a Cypher statement only!"""

correct_cypher_user = """Check for invalid syntax or semantics and return a corrected Cypher statement.

Schema:
{schema}

Note: Do not include any explanations or apologies in your responses.
Do not wrap the response in any backticks or anything else.
Respond with a Cypher statement only!

Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.

The question is:
{question}

The Cypher statement is:
{cypher}

The errors are:
{errors}

Corrected Cypher statement: """

# Correct cypher
correct_cypher_msgs = [
    (
        "system",
        correct_cypher_system,
    ),
    ("user", correct_cypher_user),
]

correct_cypher_prompt = ChatPromptTemplate.from_messages(correct_cypher_msgs)

In [15]:
async def correct_cypher(subquery, cypher, errors):
    resp = await llm_fast.achat(correct_cypher_prompt.format_messages(question=subquery, schema=schema, errors=errors))
    return resp.message.content

In [16]:
information_check_system = """You are an expert assistant that evaluates whether a set of subqueries, their results, any existing condensed information, and the current query plan provide enough details to answer a given question. Your task is to:

1. Analyze if the available information is sufficient to answer the original question: "{original_question}".
2. Review the remaining steps in the query plan (if any) to determine if they:
   - Should be retained as is.
   - Need modification to address gaps.
   - Can be skipped because the required information is already available.
   - Should be reorganized for optimized execution (e.g., parallel processing).

### Process:

#### 1. Analyze the Original Question
   - Identify the key pieces of information required to fully answer the question.
   - Break the question into smaller components if it involves multiple aspects.

#### 2. Review Available Information
   - Examine the subqueries, their results, and any provided condensed information.
   - Assess if they collectively address all components of the question.

#### 3. Identify Information Gaps
   - Compare the requirements from the question against the available information.
   - Highlight any missing details or incomplete data that must be retrieved to form a complete answer.
   - If any critical subqueries fail to produce results essential for answering the question:
     - Document the missing information in the **dynamic notebook**.
     - Mark the task as **unsolvable** due to the missing data.
     - Do not produce a modified query plan, as the question cannot be answered with the available or fetchable information.

#### 4. Update and Refine the **Dynamic Notebook**
   - Treat the condensed information as a **central knowledge base**:
     - Continuously update it with key details from subquery results.
     - Integrate new data to close gaps and establish connections between facts.
   - Ensure the notebook reflects the current state of knowledge, including any identified gaps or failed subqueries.
   - Document explicitly if the task is unsolvable due to missing information.

#### 5. Modify the Query Plan (If Solvable)
   - If sufficient information is available:
     - Generate a concise and accurate answer to the question.
     - Specify which remaining steps (if any) in the query plan can be skipped.
   - If information is insufficient but fetchable:
     - Suggest additional subqueries to retrieve the missing details.
     - Ensure new subqueries are designed specifically to fill identified gaps.
     - Organize all subqueries into parallel-executable groups wherever possible.
     - Maintain sequential steps only when strict data dependencies exist.
   - If critical gaps exist that cannot be resolved due to failed subqueries, do not modify the query plan and clearly state why the task cannot be completed.

### Key Guidelines:
- **Focus Only on Information Retrieval**: Limit query plans to fetching data and avoid reasoning/analysis tasks.
- **Optimize for Parallel Execution**: Group queries into parallelizable blocks to reduce execution time.
- **Maintain Sequential Order Only When Necessary**: Use sequential steps only when results of one query depend on another.
- **Centralize Knowledge**:
   - Use the dynamic notebook to consolidate all available information.
   - Ensure it remains the authoritative source for answering the question and guiding further steps.
   - Explicitly document unsolvable tasks if critical information is missing.
"""

information_check_user = """
Subqueries and their results:  
{subqueries}  
Existing dynamic notebook:  
{dynamic_notebook}
Current remaining plan (if any):
{plan}
Original question: {question}  
"""

information_check_msgs = [
    (
        "system",
        information_check_system,
    ),
    ("user", information_check_user),
]

information_check_prompt = ChatPromptTemplate.from_messages(information_check_msgs)

class IFOutput(BaseModel):
    """
    Represents the output of an information sufficiency evaluation process. 
    Contains either a condensed summary of the available information or additional subqueries needed to answer the original question.
    """

    dynamic_notebook: str = Field(
        description="A continuously updated and refined summary integrating subquery results and condensed information. Serves as the central knowledge base to address the original question and guide further subqueries if necessary."
    )
    modified_plan: Optional[List[List[str]]] = Field(
        description="Modified version of the remaining plan steps. Each group contains queries that can be executed in parallel. Null if no remaining plan exists, all gaps have been addressed, or the task is unsolvable due to missing critical information."
    )

In [17]:
def format_subqueries_for_prompt(information_checks: list) -> str:
    """
    Converts a list of InformationCheck objects into a string that can be added to a prompt.
    
    Args:
        information_checks (List[InformationCheck]): List of information checks to process.
    
    Returns:
        str: A formatted string representing subqueries and their results.
    """
    subqueries_and_results = []
    
    for check in information_checks:
        # Extract the first result if available, otherwise use "No result available."
        result = (
            check.database_output[0] if check.database_output else "No result available."
        )
        subqueries_and_results.append(
            f"- Subquery: {check.subquery}\n  Result: {result}"
        )
    
    return "\n".join(subqueries_and_results)

def information_check(subquery_events, original_question, dynamic_notebook, plan):
    subqueries = format_subqueries_for_prompt(subquery_events)
    print(f"Before: {dynamic_notebook}")
    print(f"Plan: {plan}")
    llm_output =   (
        llm.as_structured_llm(IFOutput)
        .complete(information_check_prompt.format(subqueries=subqueries, original_question=original_question, dynamic_notebook=dynamic_notebook, plan=plan))
        .raw
    )
    print(f"After: {llm_output.dynamic_notebook}")
    print(f"New Plan: {llm_output.modified_plan}")
    return {'dynamic_notebook': llm_output.dynamic_notebook, 'modified_plan': llm_output.modified_plan}
        

In [18]:
final_answer_system = """You are a highly intelligent assistant trained to provide concise and accurate answers. You will be given a context and a user question. Your task is to analyze the context and answer the user question based on the information provided in the context. If the context lacks sufficient information to answer the question, inform the user and suggest what additional details are needed.

Focus solely on the context to form your response. Avoid making assumptions or using external knowledge unless explicitly stated in the context.
Ensure the final answer is clear, relevant, and directly addresses the user’s question.
If the question is ambiguous, ask clarifying questions to ensure accuracy before proceeding."""

final_answer_user = """
Based on this context:
{context}

Answer the following question:
<question>
{question}
</question>

Provide your answer based on the context above explain your reasoning.
If clarification or additional information is needed, explain why and specify what is required.
"""

final_answer_msgs = [
    (
        "system",
        final_answer_system,
    ),
    ("user", final_answer_user),
]

final_answer_prompt = ChatPromptTemplate.from_messages(final_answer_msgs)

async def generate_final_answer(question, context):
    resp = await llm_fast.achat(final_answer_prompt.format_messages(question=question, context=context))
    return resp.message.content                 
    

In [19]:
MAX_INFORMATION_CHECKS = 3

class GenerateCypher(Event):
    subquery: str
    
class ValidateCypher(Event):
    subquery: str
    generated_cypher: str

class CorrectCypher(Event):
    cypher: str
    subquery: str
    errors: List[str]

class ExecuteCypher(Event):
    validated_cypher: str
    subquery: str

class InformationCheck(Event):
    cypher: str
    subquery: str
    database_output: list
    
class GenerateFinalAnswer(Event):
    context: str

class ConcurrentFlow(Workflow):
    @step
    async def start(self, ctx: Context, ev: StartEvent) -> GenerateCypher | GenerateFinalAnswer:
        original_question = ev.input
        await ctx.set("original_question", original_question) # So we don't need to pass the question all the time
        await ctx.set("information_checks", 0) # Current number of information check steps
        await ctx.set("dynamic_notebook", "") # Current knowledge
        await ctx.set("subqueries_cypher_history", {}) # History of which queries were executed
        guardrails_output = guardrails_step(original_question)
        if guardrails_output.get("next_event") == "generate_final_answer":
            context = "The question is not about movies or cast, so I cannot answer the question"
            return GenerateFinalAnswer(context=context)

        # store in global context 
        subqueries = guardrails_output["arguments"].get("plan")
        await ctx.set("count_of_subqueries", len(subqueries[0])) #we use this in ctx.collect()
        await ctx.set("plan", subqueries[1:]) #Later plan after we finish current step
        # Send events in the current step of the plan
        for subquery in subqueries[0]:
            print(subquery)
            ctx.send_event(GenerateCypher(subquery=subquery))


    @step(num_workers=4)
    async def generate_cypher_step(self, ctx: Context, ev: GenerateCypher) -> ValidateCypher:
        #print("Running generate_cypher ", ev.subquery)
        generated_cypher = await generate_cypher(ev.subquery)
        return ValidateCypher(subquery=ev.subquery, generated_cypher=generated_cypher)

    @step(num_workers=4)
    async def validate_cypher_step(self, ctx: Context, ev: ValidateCypher) -> GenerateFinalAnswer | ExecuteCypher | CorrectCypher:
        #print("Running validate_cypher ", ev)
        results = validate_cypher(ev.subquery, ev.generated_cypher)
        if results['next_action'] == "end": # DB value mapping
            return GenerateFinalAnswer(context=str(results["mapping_errors"]))
        if results['next_action'] == "execute_cypher":
            return ExecuteCypher(subquery=ev.subquery, validated_cypher=ev.generated_cypher)
        if results['next_action'] == "correct_cypher":
            return CorrectCypher(subquery=ev.subquery, cypher=ev.generated_cypher, errors=results['cypher_errors'])

    @step(num_workers=4)
    async def correct_cypher_step(self, ctx: Context, ev: CorrectCypher) -> ValidateCypher:
        print("Running validate_cypher ", ev)
        results = await correct_cypher(ev.subquery, ev.cypher, ev.errors)
        return ValidateCypher(subquery=ev.subquery, generated_cypher=results)
    
    @step
    async def execute_cypher_step(self, ctx: Context, ev: ExecuteCypher) -> InformationCheck:
        try:
            database_output = graph_store.structured_query(ev.validated_cypher)
        except Exception as e: # Dividing by zero, etc... or timeout
            database_output = e.message
        return InformationCheck(subquery=ev.subquery, cypher=ev.validated_cypher, database_output=database_output)

    @step
    async def information_check_step(self, ctx: Context, ev: InformationCheck) -> GenerateCypher | GenerateFinalAnswer:
        # wait until we receive all events
        #print("Running information_check_step", ev)
        # retrieve from context
        number_of_subqueries = await ctx.get("count_of_subqueries")
        result = ctx.collect_events(ev, [InformationCheck] * number_of_subqueries)
        if result is None:
            return None
        # Add executed cypher statements to global state
        subqueries_cypher_history = await ctx.get("subqueries_cypher_history")
        new_subqueries_cypher = {
                item.subquery: {
                    "cypher": item.cypher,
                    "database_output": item.database_output
                } for item in result
            }
        await ctx.set("subqueries_cypher_history", {**subqueries_cypher_history, **new_subqueries_cypher})

        original_question = await ctx.get("original_question")
        dynamic_notebook = await ctx.get("dynamic_notebook")
        plan = await ctx.get("plan")

        # Do the information check
        
        data = information_check(result, original_question, dynamic_notebook, plan)
        # Get count of information checks done
        information_checks = await ctx.get("information_checks")
        # Go fetch additional information if needed
        if data.get("modified_plan") and information_checks < MAX_INFORMATION_CHECKS:
            await ctx.set("count_of_subqueries", len(data['modified_plan'][0])) # this is used for ctx.collect()
            await ctx.set("dynamic_notebook", data["dynamic_notebook"])
            await ctx.set("plan", data.get("modified_plan")[1:])
            await ctx.set("information_checks", information_checks + 1)
            for subquery in data["modified_plan"][0]:
                ctx.send_event(GenerateCypher(subquery=subquery))
        else:
            return GenerateFinalAnswer(context=data['dynamic_notebook'])

    @step
    async def final_answer(self, ctx: Context, ev: GenerateFinalAnswer) -> StopEvent:
        original_question = await ctx.get("original_question")
        subqueries_cypher_history = await ctx.get("subqueries_cypher_history")
        # wait until we receive all events
        print("Running final_answer ", ev)
        resp = await generate_final_answer(original_question, ev.context)
        return StopEvent(result={"text":resp, "subqueries_cypher_history": subqueries_cypher_history})

In [20]:
"""
from llama_index.utils.workflow import (
    draw_all_possible_flows,
    draw_most_recent_execution,
)

draw_most_recent_execution(w, filename="joke_flow_recent.html")
draw_all_possible_flows(w, filename="joke_flow_recenst.html")
"""

'\nfrom llama_index.utils.workflow import (\n    draw_all_possible_flows,\n    draw_most_recent_execution,\n)\n\ndraw_most_recent_execution(w, filename="joke_flow_recent.html")\ndraw_all_possible_flows(w, filename="joke_flow_recenst.html")\n'

In [21]:
import pandas as pd
# URL of the Parquet file
url = "https://huggingface.co/datasets/tomasonjo/text2cypher_demo_test/resolve/main/test.parquet"

# Read the Parquet file directly from the URL
df = pd.read_parquet(url)

# Display the first few rows
df.head(2)

,question,schema,cypher,data_source,instance_id,database_reference_alias,question_embedding,text_answers
0,List the first 3 movies that have been directe...,Node properties:\n- **Movie**\n - `url`: STRI...,MATCH (p:Person)-[:ACTED_IN]->(m:Movie)<-[:DIR...,neo4jLabs_synthetic_claudeopus,instance_id_35354,neo4jlabs_demo_db_recommendations,"[-0.04787346348166466, 0.02376137115061283, 0....",The first three movies that have been directed...
1,List the first 5 directors who have directed a...,Node properties:\n- **Movie**\n - `url`: STRI...,MATCH (d:Director)-[:DIRECTED]->(m:Movie) WHER...,neo4jLabs_synthetic_claudeopus,instance_id_35510,neo4jlabs_demo_db_recommendations,"[-0.02948657236993313, 0.028692537918686867, 0...",The first 5 directors who have directed a movi...


In [ ]:
from datetime import datetime
import time
w = ConcurrentFlow(timeout=120, verbose=False)
answers = []
times = []

# Add latency
for i, row in df.iterrows():
    print(f"{i} at {datetime.now()}")
    time.sleep(10)
    question = row['question']
    print("#" * 30) 
    start = datetime.now()
    try:
        result = await w.run(input=question)
        answers.append(result)
        times.append(start - datetime.now())
    except:
         answers.append(None)
         times.append(start - datetime.now())

0 at 2024-12-19 11:48:04.085334
##############################
List all movies directed by actors.
Before: 
Plan: [['List the release date of each movie from the previous query.']]
After: The movie 'Mighty Aphrodite' is directed by an actor.
New Plan: [["List the release date of the movie 'Mighty Aphrodite'."], ['List the first 3 movies that have been directed by actors.']]
Before: The movie 'Mighty Aphrodite' is directed by an actor.
Plan: [['List the first 3 movies that have been directed by actors.']]
After: The movie 'Mighty Aphrodite' is directed by an actor. The release date of the movie 'Mighty Aphrodite' is 1995-11-10.
New Plan: [['List the first 3 movies that have been directed by actors.']]
Before: The movie 'Mighty Aphrodite' is directed by an actor. The release date of the movie 'Mighty Aphrodite' is 1995-11-10.
Plan: []
After: The movie 'Mighty Aphrodite' is directed by an actor. The release date of the movie 'Mighty Aphrodite' is 1995-11-10. We have found 1 movie directed

Exception in callback Dispatcher.span.<locals>.wrapper.<locals>.handle_future_result(span_id='Workflow.run...-f35fab2db11f', bound_args=<BoundArgumen...or higher.'})>, instance=<__main__.Con...t 0x352711490>, context=<_contextvars...t 0x3527117c0>)(<WorkflowHand...string_type")>) at /Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/instrumentation/dispatcher.py:273
handle: <Handle Dispatcher.span.<locals>.wrapper.<locals>.handle_future_result(span_id='Workflow.run...-f35fab2db11f', bound_args=<BoundArgumen...or higher.'})>, instance=<__main__.Con...t 0x352711490>, context=<_contextvars...t 0x3527117c0>)(<WorkflowHand...string_type")>) at /Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/instrumentation/dispatcher.py:273>
Traceback (most recent call last):
  File "/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/workflow/workflow.py", line 247, in _task
    new_ev = await instrumented_step(**kwargs)
      

2 at 2024-12-19 11:48:54.550165
##############################
List all directors and the languages of the movies they have directed.
Before: 
Plan: []
After: The current notebook contains information about directors and the languages of the movies they have directed. However, it only contains information about one director, John Lasseter, and his movies are only in English. To answer the question 'Which three directors have directed movies in more than one language?', we need to gather information about more directors and their movies' languages.
New Plan: [['List all directors and the languages of the movies they have directed.']]
Before: The current notebook contains information about directors and the languages of the movies they have directed. However, it only contains information about one director, John Lasseter, and his movies are only in English. To answer the question 'Which three directors have directed movies in more than one language?', we need to gather information about 

Exception in callback Dispatcher.span.<locals>.wrapper.<locals>.handle_future_result(span_id='Workflow.run...-91a2dfc1d4be', bound_args=<BoundArgumen...0 minutes.'})>, instance=<__main__.Con...t 0x352711490>, context=<_contextvars...t 0x35552a380>)(<WorkflowHand...string_type")>) at /Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/instrumentation/dispatcher.py:273
handle: <Handle Dispatcher.span.<locals>.wrapper.<locals>.handle_future_result(span_id='Workflow.run...-91a2dfc1d4be', bound_args=<BoundArgumen...0 minutes.'})>, instance=<__main__.Con...t 0x352711490>, context=<_contextvars...t 0x35552a380>)(<WorkflowHand...string_type")>) at /Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/instrumentation/dispatcher.py:273>
Traceback (most recent call last):
  File "/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/workflow/workflow.py", line 247, in _task
    new_ev = await instrumented_step(**kwargs)
      

7 at 2024-12-19 11:51:11.300529
##############################
Retrieve all user IDs and their corresponding movie ratings.


...

18 at 2024-12-19 11:55:59.340647
##############################
List all directors and the countries where they have directed movies.
Before: 
Plan: []
After: The current notebook contains information about directors and the countries where they have directed movies. For example, 'John Lasseter' has directed movies in 'USA'. However, this is only one entry, and we need to find directors who have directed movies in more than three different countries. The current information is insufficient to answer the question.
New Plan: [['List all directors and the countries where they have directed movies.'], ['Group the results by director and count the number of distinct countries they have directed movies in.'], ['Filter the grouped results to include only directors who have directed movies in more than three countries.'], ['List the top 5 directors from the filtered results.']]
Before: The current notebook contains information about directors and the countries where they have directed movies. 

Exception in callback Dispatcher.span.<locals>.wrapper.<locals>.handle_future_result(span_id='Workflow.run...-83b33db66ded', bound_args=<BoundArgumen...n dollars.'})>, instance=<__main__.Con...t 0x352711490>, context=<_contextvars...t 0x37a674ac0>)(<WorkflowHand...string_type")>) at /Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/instrumentation/dispatcher.py:273
handle: <Handle Dispatcher.span.<locals>.wrapper.<locals>.handle_future_result(span_id='Workflow.run...-83b33db66ded', bound_args=<BoundArgumen...n dollars.'})>, instance=<__main__.Con...t 0x352711490>, context=<_contextvars...t 0x37a674ac0>)(<WorkflowHand...string_type")>) at /Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/instrumentation/dispatcher.py:273>
Traceback (most recent call last):
  File "/Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/workflow/workflow.py", line 247, in _task
    new_ev = await instrumented_step(**kwargs)
      

24 at 2024-12-19 11:58:38.241576
##############################
What are the release dates of all movies in the USA?
What are the titles of all movies released in the USA?
Before: 
Plan: []
After: The dynamic notebook contains the following information:
- The movie 'Toy Story' was released in the USA on '1995-11-22'.
New Plan: None
Running final_answer  context="The dynamic notebook contains the following information:\n- The movie 'Toy Story' was released in the USA on '1995-11-22'."


Exception in callback Dispatcher.span.<locals>.wrapper.<locals>.handle_future_result(span_id='Workflow.run...-f9deb3eb6fa3', bound_args=<BoundArgumen...n the USA?'})>, instance=<__main__.Con...t 0x352711490>, context=<_contextvars...t 0x37c288b80>)(<WorkflowHandler cancelled>) at /Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/instrumentation/dispatcher.py:273
handle: <Handle Dispatcher.span.<locals>.wrapper.<locals>.handle_future_result(span_id='Workflow.run...-f9deb3eb6fa3', bound_args=<BoundArgumen...n the USA?'})>, instance=<__main__.Con...t 0x352711490>, context=<_contextvars...t 0x37c288b80>)(<WorkflowHandler cancelled>) at /Users/tomazbratanic/anaconda3/lib/python3.11/site-packages/llama_index/core/instrumentation/dispatcher.py:273>
Traceback (most recent call last):
  File "/Users/tomazbratanic/anaconda3/lib/python3.11/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/Users/tomazbratanic/anaconda3/lib/p

25 at 2024-12-19 11:58:58.176535
##############################


In [ ]:
df['answers'] = answers

In [ ]:
df['times'] = times

In [ ]:
df.head(2)

In [ ]:
df.to_csv("gpt-4.csv", index=False)

In [ ]:
#import pandas as pd
#df = pd.read_csv('gpt-4.csv')

In [54]:
from datasets import Dataset

questions = df['question'].to_list()
            
ground_truths = [str(el) for el in df['text_answers'].to_list()]

answers = [str(el) for el in df['answers'].to_list()]
contexts = [None] * len(questions)

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [68]:
len(answers)

203

In [67]:
len([len(el) for el in answers if len(el) > 1000])

105

In [70]:
len([len(el) for el in answers if len(el) == 4])

64

In [1]:
"""
from ragas.llms import LlamaIndexLLMWrapper

evaluator_llm = LlamaIndexLLMWrapper(OpenAI(model="gpt-4o"))
metrics = [
    Faithfulness(llm=evaluator_llm),
    AnswerRelevancy(llm=evaluator_llm),
    ContextPrecision(llm=evaluator_llm),
    ContextRecall(llm=evaluator_llm),
]
"""

'\nfrom ragas.llms import LlamaIndexLLMWrapper\n\nevaluator_llm = LlamaIndexLLMWrapper(OpenAI(model="gpt-4o"))\nmetrics = [\n    Faithfulness(llm=evaluator_llm),\n    AnswerRelevancy(llm=evaluator_llm),\n    ContextPrecision(llm=evaluator_llm),\n    ContextRecall(llm=evaluator_llm),\n]\n'

In [58]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset, 
    metrics=[
        answer_relevancy,
    ],
)

df = result.to_pandas()

Evaluating:   0%|          | 0/203 [00:00<?, ?it/s]

Exception raised in Job[4]: BadRequestError(Error code: 400 - {'error': {'message': "This model's maximum context length is 128000 tokens. However, your messages resulted in 359326 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})


KeyboardInterrupt: 

Exception raised in Job[1]: AssertionError()
Exception raised in Job[19]: AssertionError(LLM is not set)
Exception raised in Job[20]: AssertionError(LLM is not set)
Exception raised in Job[21]: AssertionError(LLM is not set)
Exception raised in Job[22]: AssertionError(LLM is not set)
Exception raised in Job[23]: AssertionError(LLM is not set)
Exception raised in Job[24]: AssertionError(LLM is not set)
Exception raised in Job[25]: AssertionError(LLM is not set)
Exception raised in Job[26]: AssertionError(LLM is not set)
Exception raised in Job[27]: AssertionError(LLM is not set)
Exception raised in Job[28]: AssertionError(LLM is not set)
Exception raised in Job[29]: AssertionError(LLM is not set)
Exception raised in Job[30]: AssertionError(LLM is not set)
Exception raised in Job[31]: AssertionError(LLM is not set)
Exception raised in Job[32]: AssertionError(LLM is not set)
Exception raised in Job[33]: AssertionError(LLM is not set)
Exception raised in Job[34]: AssertionError(LLM is not 